##### <b> 1. 개요  </b>
음성 관련 실험을 보다 편리하게 하기 위해서 음성 관련 여러가지 시각화, 분석 기법들을 보다 전문적이고, 자신이 원하는 대로 적용시키는 데 도움을 주는 프로그램을 제작하는 프로젝트

In [2]:
import os
import numpy as np
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
import scipy
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import FVA.FVA as fva
from FVA.lpc import *
from FVA.detect import *
from FVA.mfcc import *
from FVA.animation import *
saveresult = False

In [1]:
def print_2dplot(x, y, xlim, ylim, xlabel, ylabel, position, xscale = 'linear', yscale = 'linear', color = 'b', lw = 1):
    plt.subplot(position[0], position[1], position[2])
    plt.plot(x, y, color=color, lw=lw)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xscale(xscale)
    plt.yscale(yscale)

#### <b> 2. Spectrum</b>
target_sounds의 첫 번째 파일을 시각화

In [3]:
FILE_LIST = os.listdir('FVA\\target_sounds')
FILE_PATH = 'FVA\\target_sounds\\' + 'recorded.wav'

SSA = fva.SingleSoundAnalyser(FILE_PATH = FILE_PATH, duration = None)

print_2dplot(*SSA.get_source(),[2,1,1],color='slateblue')
print_2dplot(*SSA.get_spectrum(xlim=[1,5000],ylim=[1e-7,None]),[2,1,2],xscale='log',yscale='log',color='mediumslateblue',lw=0.5)

mng = plt.get_current_fig_manager()
#mng.full_screen_toggle()
if saveresult: plt.savefig('FVA\\results\\smalldrum_spectrum')
plt.show()

#### <b>3. Spectrogram</b>
target_sounds의 첫 번째 파일을 시각화

In [4]:
spec_out, fout_index, pout= SSA.get_fp_stft(True, 256, 128, 70, 8096, 3)
print(spec_out)

formants : [f1 : 40.853507905138336/f2 : 633.2293725296443/f3 : 996.8255928853754]
pitch : 0.0
formants : [f1 : 620.9733201581028/f2 : 905.5860918972331/f3 : 1203.816699604743]
pitch : 416.03773584905656
formants : [f1 : 44.93885869565217/f2 : 389.47010869565213/f3 : 800.7287549407114]
pitch : 450.0
formants : [f1 : 39.49172430830039/f2 : 588.290513833992/f3 : 867.4561511857708]
pitch : 0.0
formants : [f1 : 28.597455533596836/f2 : 514.754199604743/f3 : 890.6064723320158]
pitch : 459.375
formants : [f1 : 43.57707509881423/f2 : 531.0956027667984/f3 : 942.3542490118576]
pitch : 416.03773584905656
formants : [f1 : 42.21529150197628/f2 : 659.1032608695651/f3 : 1225.60523715415]
pitch : 424.0384615384615
formants : [f1 : 43.57707509881423/f2 : 525.6484683794466/f3 : 826.6026432806324]
pitch : 469.1489361702128
formants : [f1 : 42.21529150197628/f2 : 582.8433794466403/f3 : 887.8829051383399]
pitch : 459.375
formants : [f1 : 40.853507905138336/f2 : 705.4039031620553/f3 : 943.7160326086956]
pit

In [5]:
x = np.arange(0,  spec_out.shape[1] , 1) * SSA.df0
y = np.arange(0,  spec_out.shape[0] , 1) * SSA.dt0
X, Y = np.meshgrid(x, y)

x0=np.zeros(spec_out.shape[0] * SSA.max_num_formants)
y0=np.zeros(spec_out.shape[0] * SSA.max_num_formants)
z0=np.zeros(spec_out.shape[0] * SSA.max_num_formants)
for j in range( SSA.max_num_formants):
	for i in range( spec_out.shape[0] ):
		if fout_index[i][j] >= 0:
			x0[i + j * spec_out.shape[0]]= fout_index[i][j] * SSA.df0
			y0[i + j * spec_out.shape[0]]= SSA.dt0 * i
			z0[i + j * spec_out.shape[0]]= spec_out[i][int(fout_index[i][j])]

fig = plt.figure()
ax1 = fig.add_subplot(111,projection='3d')
ax1.view_init(elev=70,azim=310,roll=0)
ax1.set_xlabel('frequency[Hz]')
ax1.set_ylabel('time[sec]')
ax1.set_zlabel('level')
#ax1.plot_wireframe(X,Y,spec_out,alpha=0.7)
ax1.plot(x0,y0,z0,'o',color='r',ms=1,mew=5)
ax1.plot_surface(X,Y,spec_out,cmap=matplotlib.cm.coolwarm,lw=0,antialiased=False,alpha=0.4)
if saveresult: plt.savefig('FVA\\results\\smalldrum_3dformants')

fig = plt.figure()
ax2 = fig.add_subplot(111)
ax2.set_ylabel('frequency[Hz]')
ax2.set_xlabel('time[sec]')
X,Y=np.meshgrid(x,y)
ax2.pcolor(np.transpose(Y),np.transpose(X),np.transpose(spec_out))
#ax2.set_yscale('log')
ax2.plot(y0, x0 , "o", color='r', ms=2, mew=2)
print(y0)
if saveresult: plt.savefig('FVA\\results\\smalldrum_spectrogram')

mng = plt.get_current_fig_manager()
#mng.full_screen_toggle()

plt.show()

[0.00000000e+00 4.53514739e-05 9.07029478e-05 ... 3.02494331e-02
 3.02947846e-02 3.03401361e-02]


#### <b>4. LPC(Linear Prediction Coding)</b>
target_sounds의 첫 번째 파일을 시각화

In [9]:
def to_db(x):
  return 10 * np.log10(x)
x,y = SSA.get_spectrum()[:2]
y = to_db(y)
plt.figure(figsize=(12,8))
#plt.subplot(2,1,1)
plt.plot(x[::],y[::],c='lightskyblue',lw=1, alpha=1, label='FFT')
#plt.xscale('log', base=10)
plt.xlim([0,8000])
FreqPoints=8096
df0 = (SSA.sr / 2.) / FreqPoints
Y = SSA.fdata.copy()
Y -= np.hstack((Y[0],Y[:-1]))*0.8
windowed = np.hamming(Y.shape[0]) * Y
a, e = lpc(windowed,lpcOrder=80)
w, h = scipy.signal.freqz(np.sqrt(e), a, FreqPoints)
lpcspec = np.abs(h)
lpcspec[lpcspec < 1.] = 1.
#lpcspec *= (1./float(1 << ((8 * SSA.samplewidth)-1)))

loglpcspec = to_db(lpcspec)

bias = y.mean() - loglpcspec.mean()

#plt.subplot(2,1,2)
plt.plot(np.linspace(0,SSA.sr/2.,len(loglpcspec)), loglpcspec+bias, label='LPC', color='green')

f_result, i_result = formant_detect(lpcspec,df0,1)
plt.plot(f_result, loglpcspec[i_result] + bias, "o", c='r', label='formants', mew=4, ms=4)
plt.legend()

mng = plt.get_current_fig_manager()
#mng.full_screen_toggle()
if saveresult: plt.savefig('FVA\\results\\smalldrum_lpcspectrum')

plt.show()
print(f_result)

[245.12104743083003, 464.3682065217391, 891.9682559288538, 1329.100790513834, 1557.8804347826085, 1973.2244318181818, 2444.4015563241105, 2685.4372529644265, 2899.237277667984, 3597.8322628458495, 3773.5023468379445, 4698.153409090909, 5056.302495059288, 5884.266921936759, 6160.708992094861, 6430.342144268774, 6684.99567687747, 6917.860671936758, 7082.63648715415, 7290.989377470355, 7495.256916996047, 8355.904150197628, 8711.329668972332, 9039.519515810276, 9355.45331027668, 9664.578186758892, 9970.97949604743, 10273.295454545454, 10574.24962944664, 10875.203804347826]


#### <b>5. MFCC </b>

In [17]:
hop_size = 15 #ms
FFT_size = 2048
freq_min = 0
mel_filter_num = 10
sr, audio = wavfile.read(FILE_PATH)
freq_high = sr / 2

audio_framed = frame_audio(audio,FFT_size=FFT_size,hop_size=hop_size,sample_rate=sr)

window = get_window("hann",FFT_size,fftbins=True)
audio_win = audio_framed * window

audio_winT = np.transpose(audio_win)

audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

for n in range(audio_fft.shape[1]):
    audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

audio_fft = np.transpose(audio_fft)

audio_power = np.square(np.abs(audio_fft))

filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)

filters = get_filters(filter_points, FFT_size)

enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
filters *= enorm[:, np.newaxis]

audio_filtered = np.dot(filters, np.transpose(audio_power))
audio_log = 10.0 * np.log10(audio_filtered)

dct_filter_num = 40

dct_filters = dct(dct_filter_num, mel_filter_num)

cepstral_coefficents = np.dot(dct_filters, audio_log)

plt.figure(figsize=(15,5))
plt.plot(np.linspace(0, len(audio) / sr, num=len(audio)), audio)
plt.imshow(cepstral_coefficents, aspect='auto', origin='lower')

pa = PauseAnimation(np.swapaxes(cepstral_coefficents,1,0))
plt.show()

MEL min: 0.0
MEL max: 3923.337321740179


In [37]:


import matplotlib.pyplot as plt
import librosa.display
import librosa
import numpy as np
import matplotlib.cm as cm

path = 'FVA//target_sounds//2023711작은탐.wav'
sample_rate=44100

nMEL=128
nMFCC = 128
df0 = (sample_rate//2)/nMFCC
n_fft = 2048
win_length = 2048
hop_length = 1024
 
x = librosa.load(path=path,sr=sample_rate)[0]
D = np.abs(librosa.stft(x, n_fft=n_fft, win_length = win_length, hop_length=hop_length))
mel_spec = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=nMEL, hop_length=hop_length, win_length=win_length)
librosa.display.specshow(librosa.amplitude_to_db(mel_spec, ref=0.00002), sr=sr, hop_length = hop_length, y_axis='mel', x_axis='time', cmap = cm.jet)
plt.colorbar(format='%2.0f dB')
plt.show()

log_mel = librosa.power_to_db(mel_spec, ref=np.max)
mfcc = librosa.feature.mfcc(S=log_mel, n_mfcc=nMFCC)

print(mfcc.shape)

mfcc_time = np.swapaxes(mfcc,1,0)
mel_time = np.swapaxes(log_mel,1,0)
temp_formants = [formant_detect(i,df0,f_min=0) for i in mel_time]

formants_list = []
x_list = []
for i,elem in enumerate(temp_formants):
    for elem2 in elem[0][:min(5,len(elem[0]))]:
        formants_list.append(elem2)
        x_list.append(i)

plt.figure()
plt.plot(x_list,formants_list,"o",c='r',mew=1,ms=1)
plt.show()

x = np.linspace(0,sample_rate//2,nMFCC)

plt.figure()
plt.plot(x,mel_time[0])
plt.plot(temp_formants[0][0][:5],mel_time[0][temp_formants[0][1][:5]],"o", c='r', label='formants', mew=4, ms=4)
plt.show()

pa = PauseAnimation()

(128, 471)


In [36]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np


class PauseAnimation:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Click to pause/resume the animation')

        self.ax.set_ylim(mel_time.min(),mel_time.max())
        self.p, = self.ax.plot(x,mel_time[0])
        self.f, = self.ax.plot(temp_formants[0][0],mel_time[0][temp_formants[0][1]],"o")

        self.animation = animation.FuncAnimation(
            self.fig, self.update, frames=200, interval=50, blit=True)
        self.paused = True

        self.fig.canvas.mpl_connect('button_press_event', self.toggle_pause)

    def toggle_pause(self, *args, **kwargs):
        if self.paused:
            self.animation.resume()
        else:
            self.animation.pause()
        self.paused = not self.paused

    def update(self, i):
        self.p.set_ydata(mel_time[i])
        self.f.set_ydata(mel_time[i][temp_formants[i][1]])
        self.f.set_xdata(temp_formants[i][0])
        return (self.p,self.f)


pa = PauseAnimation()
plt.show()

# 여러 파일 자동 시각화
target_sounds 내의 .wav 파일을 전부 시각화하여 그 결과를 results에 이미지 형태로 저장

#### 1. '.wav' 파일 긁어오기

In [ ]:
FILE_LIST = os.listdir('FVA\\target_sounds')
WAV_LIST = []
for name in FILE_LIST:
  if os.path.splitext(name)[1] == '.wav':
    WAV_LIST.append(name)
print(WAV_LIST)

#### 2. SingleSoundAnalyser로 '.wav' 파일 불러오기

In [ ]:
DURATION = None
SAMPLE_RATE = None
SSA_LIST = []
for name in WAV_LIST:
  SSA_LIST.append(fva.SingleSoundAnalyser('FVA\\target_sounds\\'+name, DURATION, SAMPLE_RATE))

In [ ]:
print_2dplot(*SSA_LIST[0].get_source(),[1,2,1])